# Using Pre Generated Primitive for Operations

For doing any private multi-party computation in SyMPC we generally require primitives. In SyMPC we can generate these primitives in two ways:

- Generate Primitive on the fly (during the computation).
- Generate Primitive before the computation.

Author: 
- Anubhav Singh - [Twitter](https://twitter.com/aanurraj) - [GitHub](https://github.com/aanurraj)


In this tutorial we will see how to generate this primitives before performing any computation and what advantage it has over the primitive generated on the fly.

We will start with basic import for doing a multi-party computation. You can read more about these imports in introduction notebook.

In [1]:
import syft as sy

sy.logger.remove()
import torch

from sympc.session import Session
from sympc.session import SessionManager
from sympc.tensor import MPCTensor

In [2]:
# Define the virtual machines that would be use in the computation
alice_vm = sy.VirtualMachine(name="alice")
bob_vm = sy.VirtualMachine(name="bob")

# Get clients from each VM
alice = alice_vm.get_root_client()
bob = bob_vm.get_root_client()

parties = [alice, bob]

# Setup the session for the computation
session = Session(parties=parties)
SessionManager.setup_mpc(session)

Next, we will declare PyTorch tensors and will share these tensors among parties and perform all the computations on these tensors. 

In [3]:
# Define the private values to shares
x_secret = torch.randn((1000, 1000))
y_secret = torch.randn((1000, 1000))

# Share the secret between the parties
x = x_secret.share(parties=parties)
y = y_secret.share(parties=parties)

print(x)
print(y)

[MPCTensor]
Shape: torch.Size([1000, 1000])
Requires Grad: False
	| <VirtualMachineClient: alice Client> -> ShareTensorPointer
	| <VirtualMachineClient: bob Client> -> ShareTensorPointer
[MPCTensor]
Shape: torch.Size([1000, 1000])
Requires Grad: False
	| <VirtualMachineClient: alice Client> -> ShareTensorPointer
	| <VirtualMachineClient: bob Client> -> ShareTensorPointer


Lets checkout how much time a matmul operations takes to compute when primitives are generated on the fly. ⏱

In [4]:
import time

In [15]:
t1 = time.time()
x @ y
t2 = time.time()
print(t2-t1)

38.517215967178345


The above result may vary depending on the configuration of the machine on which the operation is performed.
Next, we will see how to generate the primitive before hand.

### Pre Generating Primitives

For pre generating the primitives we need to import the CryptoPrimitiveProvider, lets do that.

In [6]:
from sympc.store.crypto_primitive_provider import CryptoPrimitiveProvider

We have to follow two steps to generate the primitives.
- Log the primitive we need.
- Generate the primitives using the above generated logs.

The logs are generally the meta information needed in the computation which mainly comprises of shapes of the primitive required.

In [7]:
CryptoPrimitiveProvider.start_logging()
x @ y
log = CryptoPrimitiveProvider.stop_logging()

print(log)

{'beaver_matmul': [({'a_shape': (1000, 1000), 'b_shape': (1000, 1000)}, {'a_shape': (1000, 1000), 'b_shape': (1000, 1000), 'nr_parties': 2})]}


In [12]:
# Generate the primitives using the logs generated in the previous step.
CryptoPrimitiveProvider.generate_primitive_from_dict(primitive_log=log, session=session)

We can now calculate the time required to do the same computation using the pre-generated primitives.

In [13]:
t1 = time.time()
x @ y
t2 = time.time()
print(t2-t1)

26.111281156539917


We can clearly see the difference in the calculation time. This pre generated way comes handy when we want to do long and complex calculations.